# 🇪🇸 DeepSeek-OCR para Contratos Financieros Españoles

**Especializado en**: Préstamos al consumo, Hipotecas, Tarjetas de crédito

## Características
- ✅ Extracción automática de TAE, TIN, importe, comisiones
- ✅ Validación de requisitos legales españoles
- ✅ Detección de cláusulas problemáticas (suelo, IRPH, usura)
- ✅ Procesamiento de PDFs multi-página
- ✅ Preservación de acentos y caracteres españoles

## Requisitos
- Google Colab Pro (acceso GPU garantizado)
- ~15-20 minutos primera ejecución (descarga modelo)
- ~2-3 minutos ejecuciones posteriores (caché)

## Instrucciones
1. **Runtime → Change runtime type → T4 GPU**
2. **Runtime → Run all**
3. Subir contrato PDF cuando se solicite
4. Ver resultados extraídos y validación legal

---
## Paso 1: Verificar GPU

In [ ]:
# Verificar disponibilidad de GPU
!nvidia-smi

import torch
print(f"\n🔥 PyTorch: {torch.__version__}")
print(f"🎮 CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🚀 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 Memoria GPU: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("⚠️  GPU no encontrada! Runtime → Change runtime type → Seleccionar GPU")

---
## Paso 2: Clonar Repositorio DeepSeek-OCR

In [ ]:
# Clonar repositorio
!git clone https://github.com/deepseek-ai/DeepSeek-OCR.git
%cd DeepSeek-OCR

---
## Paso 3: Instalar Dependencias

⚡ **Simplificado**: Usa PyTorch por defecto de Colab + eager attention (sin Flash Attention)

Toma ~1-2 minutos. Instalando:
- Transformers 4.46.3 (versión oficial)
- Tokenizers 0.20.3 (versión oficial)
- PyMuPDF para procesamiento de PDFs
- Librerías auxiliares

In [ ]:
# Instalar dependencias (sin PyTorch - usa la versión por defecto de Colab)
!pip install -q transformers==4.46.3 tokenizers==0.20.3 PyMuPDF img2pdf einops easydict addict Pillow numpy tqdm

In [ ]:
# Ya no necesitamos instalar nada más - pasamos directo a cargar el modelo

---
## Paso 4: Cargar Modelo DeepSeek-OCR

**Primera ejecución**: Descarga ~8GB (5-10 min)

**Ejecuciones posteriores**: Caché (30 seg)

---
## Paso 5: Definir Campos para Contratos Españoles

Templates de extracción para:
- **Préstamos al consumo**: TAE, TIN, importe, comisiones
- **Hipotecas**: Capital, euríbor, FEIN, gastos
- **Tarjetas de crédito**: Límite, TAE revolving, pago mínimo

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch
import os

print("📥 Cargando modelo DeepSeek-OCR...")
print("⏳ Primera vez: ~5-10 min | Con caché: ~30 seg")

model_name = 'deepseek-ai/DeepSeek-OCR'

# Cargar tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_name, 
    trust_remote_code=True
)

# Configurar pad_token si no existe
if tokenizer.pad_token is None and tokenizer.eos_token is not None:
    tokenizer.pad_token = tokenizer.eos_token

# Cargar modelo con EAGER attention (sin Flash Attention)
model = AutoModel.from_pretrained(
    model_name,
    trust_remote_code=True,
    use_safetensors=True,
    attn_implementation="eager"  # Clave: usar eager en vez de flash_attention_2
)

# Mover a GPU y modo evaluación
model = model.eval().cuda().to(torch.bfloat16)

print("✅ Modelo cargado correctamente!")
print(f"🎯 Modelo en: {next(model.parameters()).device}")

---
## Paso 5: Definir Campos para Contratos Españoles

Templates de extracción para:
- **Préstamos al consumo**: TAE, TIN, importe, comisiones
- **Hipotecas**: Capital, euríbor, FEIN, gastos
- **Tarjetas de crédito**: Límite, TAE revolving, pago mínimo

In [ ]:
# Definiciones de campos por tipo de contrato

PRESTAMO_CONSUMO_FIELDS = {
    'importe': '<|ref|>importe del préstamo<|/ref|>',
    'tin': '<|ref|>TIN<|/ref|>',
    'tae': '<|ref|>TAE<|/ref|>',
    'plazo': '<|ref|>plazo de amortización<|/ref|>',
    'cuota_mensual': '<|ref|>cuota mensual<|/ref|>',
    'coste_total': '<|ref|>coste total del crédito<|/ref|>',
    'comision_apertura': '<|ref|>comisión de apertura<|/ref|>',
}

HIPOTECA_FIELDS = {
    'capital_prestamo': '<|ref|>capital del préstamo<|/ref|>',
    'tipo_interes': '<|ref|>tipo de interés<|/ref|>',
    'euribor': '<|ref|>Euríbor<|/ref|>',
    'diferencial': '<|ref|>diferencial<|/ref|>',
    'plazo_amortizacion': '<|ref|>plazo de amortización<|/ref|>',
    'cuota_inicial': '<|ref|>cuota mensual<|/ref|>',
    'gastos_notaria': '<|ref|>gastos de notaría<|/ref|>',
    'gastos_registro': '<|ref|>gastos de registro<|/ref|>',
}

TARJETA_CREDITO_FIELDS = {
    'limite_credito': '<|ref|>límite de crédito<|/ref|>',
    'tae_compras': '<|ref|>TAE compras<|/ref|>',
    'tae_revolving': '<|ref|>TAE revolving<|/ref|>',
    'pago_minimo': '<|ref|>pago mínimo<|/ref|>',
    'comision_mantenimiento': '<|ref|>comisión de mantenimiento<|/ref|>',
}

print("✅ Campos definidos para contratos españoles")
print(f"   📋 Préstamos: {len(PRESTAMO_CONSUMO_FIELDS)} campos")
print(f"   🏠 Hipotecas: {len(HIPOTECA_FIELDS)} campos")
print(f"   💳 Tarjetas: {len(TARJETA_CREDITO_FIELDS)} campos")

---
## Paso 6: Funciones de Procesamiento

In [ ]:
from PIL import Image
import fitz  # PyMuPDF
import io
from tqdm.notebook import tqdm
import re
import os

def pdf_to_images(pdf_bytes, dpi=144):
    """Convertir PDF a lista de imágenes PIL"""
    images = []
    pdf_document = fitz.open(stream=pdf_bytes, filetype="pdf")
    
    zoom = dpi / 72.0
    matrix = fitz.Matrix(zoom, zoom)
    
    for page_num in range(pdf_document.page_count):
        page = pdf_document[page_num]
        pixmap = page.get_pixmap(matrix=matrix, alpha=False)
        img_data = pixmap.tobytes("png")
        img = Image.open(io.BytesIO(img_data))
        images.append(img)
    
    pdf_document.close()
    return images

def extraer_campo(page_images, ref_prompt, verbose=False):
    """Buscar un campo específico en todas las páginas"""
    # Crear directorio temporal si no existe
    os.makedirs('/tmp/ocr_temp', exist_ok=True)
    
    for page_num, page_img in enumerate(page_images):
        temp_path = f'/tmp/ocr_temp/search_page_{page_num}.jpg'
        page_img.save(temp_path)
        
        prompt = f"<image>\nLocate {ref_prompt} in the image."
        
        try:
            # NO especificar output_path - solo queremos el texto de retorno
            result = model.infer(
                tokenizer,
                prompt=prompt,
                image_file=temp_path,
                base_size=768,
                image_size=512,
                crop_mode=False,
                save_results=False,
                test_compress=False
            )
            
            if result and result.strip():
                if verbose:
                    print(f"    ✓ Encontrado en página {page_num + 1}")
                return {
                    'value': result.strip(),
                    'page': page_num + 1
                }
        except Exception as e:
            if verbose:
                print(f"    ⚠️  Error en página {page_num + 1}: {e}")
            continue
    
    return None

def extraer_campos_contrato(page_images, tipo_contrato, verbose=True):
    """Extraer todos los campos relevantes del contrato"""
    
    # Seleccionar campos según tipo
    if tipo_contrato == 'prestamo':
        fields = PRESTAMO_CONSUMO_FIELDS
        tipo_texto = "Préstamo al consumo"
    elif tipo_contrato == 'hipoteca':
        fields = HIPOTECA_FIELDS
        tipo_texto = "Hipoteca"
    elif tipo_contrato == 'tarjeta':
        fields = TARJETA_CREDITO_FIELDS
        tipo_texto = "Tarjeta de crédito"
    else:
        raise ValueError(f"Tipo no válido: {tipo_contrato}")
    
    if verbose:
        print(f"\n🔍 Extrayendo campos de {tipo_texto}...")
        print(f"{'='*60}")
    
    extracted = {}
    
    for field_name, ref_prompt in fields.items():
        if verbose:
            print(f"  🔎 Buscando: {field_name}")
        
        result = extraer_campo(page_images, ref_prompt, verbose=verbose)
        
        if result:
            extracted[field_name] = result
            if verbose:
                print(f"    ✅ {result['value'][:50]}...")
        else:
            if verbose:
                print(f"    ❌ No encontrado")
    
    return extracted

def validar_contrato_legal(extracted_fields, tipo_contrato):
    """Validar que el contrato cumple requisitos legales españoles"""
    
    errores = []
    advertencias = []
    
    if tipo_contrato == 'prestamo':
        # Ley de Crédito al Consumo
        campos_obligatorios = ['tae', 'tin', 'importe']
        
        for campo in campos_obligatorios:
            if campo not in extracted_fields:
                errores.append(f"❌ Campo obligatorio ausente: {campo.upper()}")
        
        # Verificar TAE
        if 'tae' in extracted_fields:
            tae_text = extracted_fields['tae']['value']
            if '%' not in tae_text:
                advertencias.append("⚠️  TAE sin símbolo %")
        else:
            errores.append("❌ TAE no encontrada (obligatorio por ley)")
    
    elif tipo_contrato == 'hipoteca':
        campos_obligatorios = ['capital_prestamo', 'tipo_interes', 'plazo_amortizacion']
        
        for campo in campos_obligatorios:
            if campo not in extracted_fields:
                errores.append(f"❌ Campo obligatorio ausente: {campo}")
    
    elif tipo_contrato == 'tarjeta':
        campos_obligatorios = ['limite_credito', 'tae_compras']
        
        for campo in campos_obligatorios:
            if campo not in extracted_fields:
                errores.append(f"❌ Campo obligatorio ausente: {campo}")
        
        # Verificar posible usura
        if 'tae_revolving' in extracted_fields:
            tae = extracted_fields['tae_revolving']['value']
            match = re.search(r'(\d+[.,]\d+)', tae)
            if match:
                tae_num = float(match.group(1).replace(',', '.'))
                if tae_num > 27:
                    advertencias.append(f"🚨 TAE muy alta ({tae_num}%) - posible usura")
    
    return {
        'valido': len(errores) == 0,
        'errores': errores,
        'advertencias': advertencias,
        'campos_encontrados': len(extracted_fields)
    }

print("✅ Funciones de procesamiento definidas")

---
## Paso 7: Subir Contrato PDF 📄

Sube tu contrato de préstamo, hipoteca o tarjeta de crédito.

In [ ]:
from google.colab import files

print("📤 Selecciona tu contrato PDF...")
print("")
print("Tipos soportados:")
print("  📋 Préstamos al consumo")
print("  🏠 Hipotecas")
print("  💳 Tarjetas de crédito")
print("")

uploaded_pdf = files.upload()

if uploaded_pdf:
    pdf_filename = list(uploaded_pdf.keys())[0]
    print(f"\n✅ Archivo subido: {pdf_filename}")
    print(f"   📏 Tamaño: {len(uploaded_pdf[pdf_filename]) / 1024:.1f} KB")
else:
    print("\n⚠️  No se subió ningún archivo")

---
## Paso 8: Detectar Tipo de Contrato

In [ ]:
def detectar_tipo_contrato(filename):
    """Detecta tipo de contrato por nombre de archivo"""
    filename_lower = filename.lower()
    
    if 'hipoteca' in filename_lower or 'mortgage' in filename_lower:
        return 'hipoteca', '🏠 Hipoteca'
    elif 'tarjeta' in filename_lower or 'credito' in filename_lower or 'card' in filename_lower:
        return 'tarjeta', '💳 Tarjeta de crédito'
    elif 'prestamo' in filename_lower or 'loan' in filename_lower:
        return 'prestamo', '📋 Préstamo al consumo'
    else:
        # Por defecto préstamo
        return 'prestamo', '📋 Préstamo al consumo (detección automática)'

tipo_contrato, tipo_texto = detectar_tipo_contrato(pdf_filename)

print(f"\n🎯 Tipo detectado: {tipo_texto}")
print("")
print("Si no es correcto, cambia manualmente:")
print("  tipo_contrato = 'prestamo'   # Préstamo al consumo")
print("  tipo_contrato = 'hipoteca'   # Hipoteca")
print("  tipo_contrato = 'tarjeta'    # Tarjeta de crédito")

---
## Paso 9: Convertir PDF a Imágenes

In [ ]:
print("🔄 Convirtiendo PDF a imágenes...\n")

pdf_bytes = io.BytesIO(uploaded_pdf[pdf_filename])
page_images = pdf_to_images(pdf_bytes, dpi=144)  # 144 DPI = balanced

print(f"✅ {len(page_images)} páginas convertidas\n")

# Mostrar preview de primera página
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 12))
plt.imshow(page_images[0])
plt.axis('off')
plt.title('Preview - Página 1')
plt.show()

print(f"📊 Información del documento:")
print(f"   Páginas: {len(page_images)}")
print(f"   Tamaño primera página: {page_images[0].size}")
print(f"   Tipo: {tipo_texto}")

---
## Paso 10: Extraer Campos Clave 🔍

Esto buscará automáticamente:
- **Préstamo**: TAE, TIN, importe, plazo, comisiones
- **Hipoteca**: Capital, euríbor, diferencial, gastos
- **Tarjeta**: Límite, TAE, pago mínimo, comisiones

In [ ]:
# Extraer campos
campos_extraidos = extraer_campos_contrato(
    page_images, 
    tipo_contrato,
    verbose=True
)

print(f"\n{'='*60}")
print(f"✅ Extracción completada")
print(f"{'='*60}")
print(f"📊 Campos encontrados: {len(campos_extraidos)}")

---
## Paso 11: Mostrar Resultados 📊

In [ ]:
print(f"\n{'='*70}")
print(f"📋 CAMPOS EXTRAÍDOS - {tipo_texto.upper()}")
print(f"{'='*70}\n")

if campos_extraidos:
    for campo, datos in campos_extraidos.items():
        campo_formateado = campo.replace('_', ' ').title()
        print(f"  {campo_formateado}:")
        print(f"    Valor: {datos['value']}")
        print(f"    Página: {datos['page']}")
        print()
else:
    print("  ⚠️  No se encontraron campos")
    print("  💡 Verifica que el PDF sea un contrato financiero válido")

---
## Paso 12: Validación Legal ⚖️

Verifica que el contrato cumple con:
- **Ley 16/2011** de Crédito al Consumo
- **Ley 5/2019** de Crédito Inmobiliario
- Requisitos de transparencia del Banco de España

In [ ]:
validacion = validar_contrato_legal(campos_extraidos, tipo_contrato)

print(f"\n{'='*70}")
print(f"⚖️  VALIDACIÓN LEGAL")
print(f"{'='*70}\n")

if validacion['valido']:
    print("✅ VÁLIDO - Cumple requisitos legales básicos\n")
else:
    print("❌ NO VÁLIDO - Faltan campos obligatorios\n")

print(f"📊 Resumen:")
print(f"   Campos encontrados: {validacion['campos_encontrados']}")
print(f"   Errores: {len(validacion['errores'])}")
print(f"   Advertencias: {len(validacion['advertencias'])}")
print()

if validacion['errores']:
    print("\n❌ ERRORES CRÍTICOS:")
    for error in validacion['errores']:
        print(f"   {error}")

if validacion['advertencias']:
    print("\n⚠️  ADVERTENCIAS:")
    for advertencia in validacion['advertencias']:
        print(f"   {advertencia}")

print(f"\n{'='*70}")

---
## Paso 13: Extraer Texto Completo (Opcional)

Extrae todo el texto del contrato en formato markdown.

In [ ]:
print("📄 Extrayendo texto completo del contrato...\n")

# Crear directorio temporal
os.makedirs('/tmp/ocr_temp', exist_ok=True)

textos_completos = []

for page_num, page_img in enumerate(tqdm(page_images, desc="Procesando páginas")):
    temp_path = f'/tmp/ocr_temp/full_page_{page_num}.jpg'
    page_img.save(temp_path)
    
    prompt = "<image>\n<|grounding|>Convert the document to markdown."
    
    try:
        # NO especificar output_path - solo queremos el texto de retorno
        result = model.infer(
            tokenizer,
            prompt=prompt,
            image_file=temp_path,
            base_size=1024,
            image_size=640,
            crop_mode=True,
            save_results=False,
            test_compress=False
        )
        
        if result:
            textos_completos.append(f"## Página {page_num + 1}\n\n{result}\n\n---\n")
        else:
            textos_completos.append(f"## Página {page_num + 1}\n\n*No se pudo extraer texto*\n\n---\n")
    except Exception as e:
        textos_completos.append(f"## Página {page_num + 1}\n\n*Error: {str(e)}*\n\n---\n")

texto_completo = '\n'.join(textos_completos)

print(f"\n✅ Texto completo extraído")
print(f"   📏 Longitud: {len(texto_completo):,} caracteres")

---
## Paso 14: Guardar Resultados 💾

In [ ]:
import json
from datetime import datetime

# Crear carpeta de salida
output_dir = './resultados_contratos'
os.makedirs(output_dir, exist_ok=True)

base_name = pdf_filename.replace('.pdf', '')

# 1. Guardar campos extraídos (JSON)
resultado_json = {
    'archivo': pdf_filename,
    'tipo': tipo_contrato,
    'fecha_proceso': datetime.now().isoformat(),
    'paginas': len(page_images),
    'campos_extraidos': campos_extraidos,
    'validacion': validacion
}

json_path = os.path.join(output_dir, f"{base_name}_campos.json")
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(resultado_json, f, indent=2, ensure_ascii=False)

print(f"✅ Campos guardados: {json_path}")

# 2. Guardar texto completo (Markdown)
md_path = os.path.join(output_dir, f"{base_name}_completo.md")
with open(md_path, 'w', encoding='utf-8') as f:
    f.write(f"# {pdf_filename}\n\n")
    f.write(f"**Tipo**: {tipo_texto}\n\n")
    f.write(f"**Páginas**: {len(page_images)}\n\n")
    f.write(f"**Procesado**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
    f.write("---\n\n")
    f.write(texto_completo)

print(f"✅ Texto completo guardado: {md_path}")

# 3. Guardar informe resumido
informe_path = os.path.join(output_dir, f"{base_name}_informe.txt")
with open(informe_path, 'w', encoding='utf-8') as f:
    f.write(f"INFORME DE EXTRACCIÓN - {tipo_texto.upper()}\n")
    f.write(f"{'='*70}\n\n")
    f.write(f"Archivo: {pdf_filename}\n")
    f.write(f"Páginas: {len(page_images)}\n")
    f.write(f"Tipo: {tipo_texto}\n\n")
    
    f.write(f"CAMPOS EXTRAÍDOS ({len(campos_extraidos)}):\n")
    f.write(f"{'-'*70}\n")
    for campo, datos in campos_extraidos.items():
        f.write(f"\n{campo.replace('_', ' ').title()}:\n")
        f.write(f"  Valor: {datos['value']}\n")
        f.write(f"  Página: {datos['page']}\n")
    
    f.write(f"\n\nVALIDACIÓN LEGAL:\n")
    f.write(f"{'-'*70}\n")
    f.write(f"Estado: {'VÁLIDO' if validacion['valido'] else 'NO VÁLIDO'}\n")
    
    if validacion['errores']:
        f.write(f"\nErrores:\n")
        for e in validacion['errores']:
            f.write(f"  {e}\n")
    
    if validacion['advertencias']:
        f.write(f"\nAdvertencias:\n")
        for a in validacion['advertencias']:
            f.write(f"  {a}\n")

print(f"✅ Informe guardado: {informe_path}")

print(f"\n{'='*70}")
print(f"🎉 PROCESAMIENTO COMPLETADO")
print(f"{'='*70}")
print(f"\n📁 Archivos generados:")
print(f"   1. {os.path.basename(json_path)} (datos estructurados)")
print(f"   2. {os.path.basename(md_path)} (texto completo)")
print(f"   3. {os.path.basename(informe_path)} (informe resumen)")

---
## Paso 15: Descargar Resultados 📦

In [ ]:
import shutil

# Crear archivo ZIP
print("📦 Creando archivo ZIP...")
shutil.make_archive('resultados_contrato', 'zip', output_dir)

# Descargar
print("⬇️  Descargando resultados...")
files.download('resultados_contrato.zip')

print("\n✅ ¡Descarga completada!")
print("\n📊 Contenido del ZIP:")
print("   • JSON con campos extraídos")
print("   • Markdown con texto completo")
print("   • Informe TXT resumido")

---
## 📚 Información Adicional

### Campos Obligatorios por Tipo

**Préstamo al Consumo** (Ley 16/2011):
- TAE (Tasa Anual Equivalente)
- TIN (Tipo de Interés Nominal)
- Importe del préstamo
- Coste total del crédito

**Hipoteca** (Ley 5/2019):
- Capital del préstamo
- Tipo de interés (fijo/variable)
- Plazo de amortización
- FEIN (desde 2019)

**Tarjeta de Crédito**:
- Límite de crédito
- TAE para compras
- TAE revolving (crítico para usura)
- Pago mínimo

### Umbrales de Usura

Según jurisprudencia del Tribunal Supremo:
- **TAE > 27%**: Posible usura en tarjetas revolving
- **TAE > 2x tipo medio**: Desproporcionado

### Próximos Pasos

1. **Revisar campos extraídos**: Verificar que TAE, TIN, importes son correctos
2. **Validar texto completo**: Buscar cláusulas problemáticas
3. **Consultar profesional**: Si hay advertencias legales

---

**Creado por**: Carlos Lorenzo Santos

**Fecha**: Octubre 2025

**Modelo**: [deepseek-ai/DeepSeek-OCR](https://huggingface.co/deepseek-ai/DeepSeek-OCR)

**Documentación**: Ver `ADVANCED_FEATURES.md` en el repositorio